<a href="https://colab.research.google.com/github/tanvirrahmanaz/Topic_modeling/blob/main/Rnn_model_telecom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import data set
! gdown --id 1JrDWooV51TLnO4lGGQNwmLWQCpO9aDQB
#import libbary
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1JrDWooV51TLnO4lGGQNwmLWQCpO9aDQB
To: /content/concatenated_file.csv
100% 1.20M/1.20M [00:00<00:00, 120MB/s]


In [ ]:
!pip install bnlp_toolkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.8 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=154c1253bf0f0775816bc741337f3b1dd60d0c3e6bad1258dae39d90f219150b
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [ ]:
from bnlp import CleanText

clean_text = CleanText(
   fix_unicode=True,
   unicode_norm=True,
   unicode_norm_form="NFKC",
   remove_url=True,
   remove_email=True,
   remove_emoji=True,
   remove_number=True,
   remove_digits=True,
   remove_punct=True,
   replace_with_url="<URL>",
   replace_with_email="<EMAIL>",
   replace_with_number="<NUMBER>",
   replace_with_digit="<DIGIT>",
   replace_with_punct = "<PUNC>"
)

input_text = "আমার সাদা , মনে কোন কালো দাগ নেই,বুঝছো?<🤣"
clean_text = clean_text(input_text)
print(clean_text)

In [ ]:
from bnlp import BengaliCorpus as corpus

print(corpus.stopwords)
print(corpus.punctuations)
print(corpus.letters)
print(corpus.digits)
print(corpus.vowels)

['অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি', 'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে', 'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি', 'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই', 'একটি', 'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত', 'এতটাই', 'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা', 'এল', 'এস', 'এসে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই', 'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা', 'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে', 'করবেন', 'করলে', 'করলেন', 'করা', 'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে', 'করেই', 'করেছিলেন', 'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে', 'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই', 'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও'

In [ ]:
df = pd.read_csv("/content/concatenated_file.csv")

In [ ]:
print(df.head(4)), df["labels"].value_counts()

                                       comments labels
0                  Network er jei obostha r sim    CNI
1            23 টাকায় ৩ জিবি কত বার নেওয়া যাবে?     DP
2  কারেন্ট চলে গেলে আপনাদের নেটওয়ার্ক চলে যায়    CNI
3     01521443719 সিমে নিবন্ধন সমস্যা , কি করি?     PR


(None,
 CNI     1961
 PR      1887
 DP      1376
 NS       634
 AV       513
 IS       214
 MFS      180
 CCI       62
 pr         1
 CNi        1
 cNI        1
 CCNI       1
 Name: labels, dtype: int64)

#Apply Regular Expression

In [ ]:
# Define regex pattern
pattern = r'[^\w\s]'

# Apply regex pattern to text column
df["comments"] = df['comments'].str.replace(pattern, '')

In [ ]:
df

,comments,labels
0,Network er jei obostha r sim,CNI
1,23 টকয় ৩ জব কত বর নওয় যব,DP
2,করনট চল গল আপনদর নটওযরক চল যয,CNI
3,01521443719 সম নবনধন সমসয ক কর,PR
4,করনট চল গল নট থক ন তর আবর অফর নব,CNI
...,...,...
6826,আমর একট গরমন থরজ সম আছ সট গরমন কসটমর কযর রপলস ...,MFS
6827,5gb 130 minutes 130 taka \nTk nice net minute ...,DP
6828,কসটমর সরভস পতও পরচর সমসযয পডত হয়,CNI
6829,নটওয়রকর বরতমন অবসথ খভই খরপ ৷৷,CCNI


#Labels Value

In [ ]:
labls = ['CNI','PR','DP','NS','AV','IS','MFS','CCI']
# Remove specified values from the "labels" column
df = df.loc[df['labels'].isin(labls)]

In [ ]:
df['labels'].value_counts()

CNI    1961
PR     1887
DP     1376
NS      634
AV      513
IS      214
MFS     180
CCI      62
Name: labels, dtype: int64

#Apply Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["comments"])

In [ ]:
x = tokenizer.texts_to_sequences(df["comments"])

In [ ]:
x[1] , df["comments"][0]

([715, 95, 157, 23, 47, 152, 349, 78], 'Network er jei obostha r sim')

In [ ]:
type(x)

list

In [ ]:
len(tokenizer.index_word)

9425

In [ ]:
x = pad_sequences(x)

In [ ]:
type(x)

list

In [ ]:
x[0] , x.shape

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   73,   69,
        1630, 2242,  249,   33], dtype=int32),
 (6827, 125))

#Convert Labels into Number

In [ ]:
label_dict = {}
unique_labels = df['labels'].unique()
for index, label in enumerate(unique_labels):
    label_dict[label] = index

df['numerical_label'] = df['labels'].map(label_dict)

In [ ]:
unique_labels = df['labels'].unique()
unique_labels

array(['CNI', 'DP', 'PR', 'AV', 'MFS', 'NS', 'CCI', 'IS'], dtype=object)

In [ ]:
df, unique_labels

(                                               comments labels  \
 0                          Network er jei obostha r sim    CNI   
 1                              23 টকয় ৩ জব কত বর নওয় যব     DP   
 2                         করনট চল গল আপনদর নটওযরক চল যয    CNI   
 3                       01521443719 সম নবনধন সমসয  ক কর     PR   
 4                      করনট চল গল নট থক ন তর আবর অফর নব    CNI   
 ...                                                 ...    ...   
 6825                       সটডনট সম ক নতন বকর অফ কর দসন     PR   
 6826  আমর একট গরমন থরজ সম আছ সট গরমন কসটমর কযর রপলস ...    MFS   
 6827  5gb 130 minutes 130 taka \nTk nice net minute ...     DP   
 6828                   কসটমর সরভস পতও পরচর সমসযয পডত হয়    CNI   
 6830             Grameen thika Airtel a aste koto lagbe     PR   
 
       numerical_label  
 0                   0  
 1                   1  
 2                   0  
 3                   2  
 4                   0  
 ...               ...  
 6825              

In [ ]:
df["numerical_label"].value_counts(),df['labels'].value_counts()

(0    1961
 2    1887
 1    1376
 5     634
 3     513
 7     214
 4     180
 6      62
 Name: numerical_label, dtype: int64,
 CNI    1961
 PR     1887
 DP     1376
 NS      634
 AV      513
 IS      214
 MFS     180
 CCI      62
 Name: labels, dtype: int64)

In [ ]:
  #x == padding comment
  y = df["numerical_label"]

#create LSTM model

In [ ]:
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=x.shape[1]),
    tf.keras.layers.LSTM(units=128),  # LSTM layer
    tf.keras.layers.Dense(units=len(label_dict), activation='softmax')
])
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import datetime
log_dir = "logs/lstm_final/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=1)
model_lstm.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_val, y_val), callbacks=[tensorboard_callback])

#Test Accuracy

In [ ]:
accuracy = model_lstm.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy[1]*100))

33/33 [==============================] - 5s 66ms/step - loss: 2.8372 - accuracy: 0.5551
Accuracy: 55.51


#SAve this lstm model

In [ ]:
import pickle

# Assuming 'model' is your trained model
model_filename = 'lstm9855.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(model_lstm, model_file)
model_lstm.save_weights("lstm9855.weights.h5")


#Create TensorBoard Link

In [ ]:
!tensorboard dev upload \
  --logdir "/content/logs/lstm_final" \
  --name "model_lstm" \
  --description "using lstm" \
  --one_shot

2023-09-10 13:59:31.584975: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/vhyPPpdYQcyPoJ7sQCqzRQ/

[2023-09-10T13:59:34] Started scanning logdir.
[2023-09-10T13:59:35] Total uploaded: 300 scalars, 300 tensors (215.3 kB), 1 binary objects (223.4 kB)
[2023-09-10T13:59:35] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/vhyPPpdYQcyPoJ7sQCqzRQ/


#Using LSTM and DropOut model

In [ ]:
model_lstmD = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=300, input_length=x.shape[1]),
    tf.keras.layers.LSTM(units=128),  # You can try LSTM or GRU
    tf.keras.layers.Dropout(0.5),  # Dropout for regularization
    tf.keras.layers.Dense(units=len(label_dict), activation='softmax')
])

model_lstmD.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train-test split
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=8)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=8)


log_dir = "logs/lstmD/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


history = model_lstmD.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), callbacks=[tensorboard_callback])

Epoch 1/50
150/150 [==============================] - 65s 416ms/step - loss: 1.4943 - accuracy: 0.4546 - val_loss: 1.1464 - val_accuracy: 0.6113
Epoch 2/50
150/150 [==============================] - 60s 397ms/step - loss: 0.9824 - accuracy: 0.6622 - val_loss: 1.0262 - val_accuracy: 0.6689
Epoch 3/50
150/150 [==============================] - 52s 345ms/step - loss: 0.6825 - accuracy: 0.7683 - val_loss: 1.1411 - val_accuracy: 0.6367
Epoch 4/50
150/150 [==============================] - 48s 322ms/step - loss: 0.4824 - accuracy: 0.8428 - val_loss: 1.2212 - val_accuracy: 0.6221
Epoch 5/50
150/150 [==============================] - 49s 329ms/step - loss: 0.3657 - accuracy: 0.8889 - val_loss: 1.2907 - val_accuracy: 0.6348
Epoch 6/50
150/150 [==============================] - 49s 326ms/step - loss: 0.2642 - accuracy: 0.9194 - val_loss: 1.5188 - val_accuracy: 0.6094
Epoch 7/50
150/150 [==============================] - 50s 334ms/step - loss: 0.2145 - accuracy: 0.9341 - val_loss: 1.5124 - val_ac

#check model accuracy

In [ ]:
accuracy = model_lstmD.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy[1]*100))

33/33 [==============================] - 3s 81ms/step - loss: 3.9795 - accuracy: 0.5356
Accuracy: 53.56


#Save this lstm and dropout model

In [ ]:
import pickle

# Assuming 'model' is your trained model
model_filename = 'lstmdrop9856bv65.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(model_lstmD, model_file)
model_lstmD.save_weights("lstmdrop9856bv65.weights.h5")


#TensorBoard Link

In [ ]:
!tensorboard dev upload \
  --logdir "/content/logs/lstmD" \
  --name "simplernn" \
  --description "simple model " \
  --one_shot

2023-09-10 14:43:02.214231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/mRJboYRpSXC1fxo5Kl7pmQ/

[2023-09-10T14:43:04] Started scanning logdir.
[2023-09-10T14:43:05] Total uploaded: 300 scalars, 300 tensors (215.3 kB), 1 binary objects (227.7 kB)
[2023-09-10T14:43:05] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/mRJboYRpSXC1fxo5Kl7pmQ/


#Create Simple RNN model

In [ ]:
model_simplernnd = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=300, input_length=x.shape[1]),
    tf.keras.layers.SimpleRNN(units=128),  # SimpleRNN layer
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=len(label_dict), activation='softmax')
])

model_simplernnd.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train-test split
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=8)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=8)


log_dir = "logs/simplrnnD/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


history = model_simplernnd.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), callbacks=[tensorboard_callback])

Epoch 1/100
150/150 [==============================] - 20s 118ms/step - loss: 1.6481 - accuracy: 0.3937 - val_loss: 1.3123 - val_accuracy: 0.5537
Epoch 2/100
150/150 [==============================] - 19s 127ms/step - loss: 0.9870 - accuracy: 0.6722 - val_loss: 1.2540 - val_accuracy: 0.5908
Epoch 3/100
150/150 [==============================] - 17s 115ms/step - loss: 0.4966 - accuracy: 0.8487 - val_loss: 1.3695 - val_accuracy: 0.5615
Epoch 4/100
150/150 [==============================] - 18s 123ms/step - loss: 0.2953 - accuracy: 0.9194 - val_loss: 1.7596 - val_accuracy: 0.4668
Epoch 5/100
150/150 [==============================] - 17s 116ms/step - loss: 0.5119 - accuracy: 0.8437 - val_loss: 1.6814 - val_accuracy: 0.5479
Epoch 6/100
150/150 [==============================] - 17s 112ms/step - loss: 0.1637 - accuracy: 0.9567 - val_loss: 1.7607 - val_accuracy: 0.5371
Epoch 7/100
150/150 [==============================] - 17s 113ms/step - loss: 0.1161 - accuracy: 0.9709 - val_loss: 1.8288 -

#check model Accuracy

In [ ]:
accuracy = model_simplernnd.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy[1]*100))

33/33 [==============================] - 1s 18ms/step - loss: 3.4254 - accuracy: 0.4829
Accuracy: 48.29


#save this model

In [ ]:
import pickle

# Assuming 'model' is your trained model
model_filename = 'simplernn.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(model_simplernnd, model_file)
model_simplernnd.save_weights("simplernn.weights.h5")


#TensorBoard link

In [ ]:
!tensorboard dev upload \
  --logdir "/content/logs/fit2" \
  --name "simplernn" \
  --description "simple model " \
  --one_shot

#Only simple rnn use

In [ ]:
model_simplernn = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=300, input_length=x.shape[1]),
    tf.keras.layers.SimpleRNN(units=128),  # SimpleRNN layer
    tf.keras.layers.Dense(units=len(label_dict), activation='softmax')
])

model_simplernn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train-test split
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=8)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=8)


log_dir = "logs/fit2/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


history = model_simplernn.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val), callbacks=[tensorboard_callback])

#check Accuracy

In [ ]:
accuracy = model_simplernn.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy[1]*100))

#save this model


In [ ]:
import pickle

# Assuming 'model' is your trained model
model_filename = 'lstmdrop9856bv65.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(model_simplernnd, model_file)
model.save_weights("lstmdrop9856bv65.weights.h5")


#TensorBoard link

In [ ]:
!tensorboard dev upload \
  --logdir "/content/logs/fit2" \
  --name "simplernn" \
  --description "simple model " \
  --one_shot

#Model testing place

In [ ]:
# Sample text data
text_data = ["টাকা কেটে নিয়া যায়"]

# Tokenize and preprocess the text data
x = tokenizer.texts_to_sequences(text_data)
padded_x = pad_sequences(x, maxlen=125, padding='post', truncating='post')

# Make predictions using the model
predictions = model.predict(padded_x)

# Assuming 'predictions' contains the probabilities for each class
# You can convert it to class labels or do any post-processing you need
predicted_labels = [unique_labels[np.argmax(pred)] for pred in predictions]

# Print the predicted labels
print(predicted_labels)
predictions

1/1 [==============================] - 0s 23ms/step
['AV']


array([[0.11510694, 0.12558587, 0.10400359, 0.24369948, 0.12488133,
        0.10449511, 0.06441039, 0.11781734]], dtype=float32)

In [ ]:
! gdown --id 1x2Egg1g3UoHWInk1SBvCoDbDu0GLB7Nq

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1x2Egg1g3UoHWInk1SBvCoDbDu0GLB7Nq
To: /content/IMRAN TEST.csv
100% 178k/178k [00:00<00:00, 87.3MB/s]


In [ ]:
df_test = pd.read_csv("/content/IMRAN TEST.csv")
# Define regex pattern
pattern = r'[^\w\s]'

# Apply regex pattern to text column
df_test["comments"] = df_test['comments'].str.replace(pattern, '')

labls = ['CNI','PR','DP','NS','AV','IS','MFS','CCI']
# Remove specified values from the "labels" column
df_test = df_test.loc[df_test['labels'].isin(labls)]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_test["comments"])

x = tokenizer.texts_to_sequences(df_test["comments"])

from tensorflow.keras.preprocessing.sequence import pad_sequences



label_dict = {}
unique_labels = df_test['labels'].unique()
for index, label in enumerate(unique_labels):
    label_dict[label] = index

df_test['numerical_label'] = df_test['labels'].map(label_dict)

padded_test_data = pad_sequences(x, maxlen=125, padding='post', truncating='post')
y = df_test["numerical_label"]


<ipython-input-34-26927fd6bc7e>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test["comments"] = df_test['comments'].str.replace(pattern, '')


In [ ]:
df_test = pd.read_csv("/content/IMRAN TEST.csv")

In [ ]:
test_predictions = model.predict(padded_test_data)

# Convert the predicted numerical labels back to their original class labels
predicted_labels = [unique_labels[np.argmax(pred)] for pred in test_predictions]

# Calculate the accuracy of the model on the test data
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y, predicted_labels)

print(f"Test Accuracy: {accuracy * 100:.2f}%")

28/28 [==============================] - 4s 128ms/step
Test Accuracy: 0.00%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:227: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


In [ ]:
df['labels'] = '__label__' + df['labels'].astype(str)
df['comments_label'] = df['labels'] + ' ' + df['comments']
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3)
train.to_csv("comments.train", columns=["comments_label"], index=False, header=False)
test.to_csv("comments.test", columns=["comments_label"], index=False, header=False)

import fasttext

model = fasttext.train_supervised(input="/content/comments.train")
model.test("/content/comments.test")

<ipython-input-88-9f82f17e4ac7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'] = '__label__' + df['labels'].astype(str)
<ipython-input-88-9f82f17e4ac7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comments_label'] = df['labels'] + ' ' + df['comments']


(2027, 0.6472619634928466, 0.6472619634928466)

In [ ]:
model.test("/content/comments.test")

(2027, 0.6472619634928466, 0.6472619634928466)

In [ ]:
model.predict(padded_test_data)

AttributeError: ignored

In [ ]:
model.save_model("fasttext64.bin")


In [ ]:
import fasttext
new_comments_data = pd.read_csv("/content/IMRAN TEST.csv")
# Load your pretrained model
model = fasttext.load_model("/content/fasttext64.bin")

# Preprocess your new comments data (replace 'new_comments_data' with your actual data)
new_comments_data['labels'] = '__label__' + new_comments_data['labels'].astype(str)
new_comments_data['comments_label'] = new_comments_data['labels'] + ' ' + new_comments_data['comments']

# Load and preprocess the new comments data
new_comments_data = open("new_comments_data.txt", "r").readlines()

# Remove newline characters and split into labels and comments
new_comments = [line.strip().split(' ', 1)[1] for line in new_comments_data]

# Predict labels for the new comments
predicted_labels = [model.predict(comment)[0][0] for comment in new_comments]

# Now you have the predicted labels for your new comments

# Calculate accuracy if you have ground truth labels for the new data
# Assuming you have the true labels for the new comments in 'true_labels'
accuracy = (np.array(predicted_labels) == np.array(true_labels)).mean()
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 9.90%


In [ ]:
# Assuming you have prepared your new comments data in 'new_comments_data.txt'
# Each line in the file should be in the format: '__label__LABEL text_of_comment'

# Load the trained FastText model
model = fasttext.load_model("your_trained_model.bin")

# Load and preprocess the new comments data
new_comments_data = open("new_comments_data.txt", "r").readlines()

# Remove newline characters and split into labels and comments
new_comments = [line.strip().split(' ', 1)[1] for line in new_comments_data]

# Predict labels for the new comments
predicted_labels = [model.predict(comment)[0][0] for comment in new_comments]

# Now you have the predicted labels for your new comments

# Calculate accuracy if you have ground truth labels for the new data
# Assuming you have the true labels for the new comments in 'true_labels'
accuracy = (np.array(predicted_labels) == np.array(true_labels)).mean()
print(f"Accuracy: {accuracy * 100:.2f}%")
